# <center style='color:deeppink'>Calculate `Inception Score (IS)` using PyTorch</center>

# 1. Import required libraries

In [1]:
import torch
print('PyTorch version:', torch.__version__)
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

import torchvision
print('Torchvision version:', torchvision.__version__)
from torchvision import datasets, transforms
from torchvision import models
from torchvision.models.inception import inception_v3

import os
import pathlib
from PIL import Image 
import numpy as np
from scipy.stats import entropy

PyTorch version: 2.9.0+cpu
Torchvision version: 0.24.0+cpu


# 2. Define `Inception` function

In [2]:
def calculate_inception_score(imgs, batch_size, resize, splits):
    
    N = len(imgs)
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    dataloader = DataLoader(imgs, batch_size=batch_size, shuffle=False)
    
    inception_model = inception_v3(weights=models.Inception_V3_Weights.DEFAULT, transform_input=False).to(device)
    inception_model.eval()
    up = nn.Upsample(size=(299, 299), mode='bilinear').to(device)
    
    def get_pred(x):
        if resize:
            x = up(x)
        x = inception_model(x)
        return F.softmax(x, dim=1).detach().cpu().numpy()
    
    # get predictions
    preds = np.zeros((N, 1000))
    
    for i, j in enumerate(dataloader):
        batch = j.to(device)
        batch_size_i = batch.size()[0]
        preds[i*batch_size:i*batch_size + batch_size_i] = get_pred(batch)
        
    # compute the mean kl-div
    split_scores = []
    
    for k in range(splits):
        part = preds[k * (N // splits): (k+1) * (N // splits), :]
        py = np.mean(part, axis=0)
        scores = []
        for i in range(part.shape[0]):
            pyx = part[i, :]
            scores.append(entropy(pyx, py))
        split_scores.append(np.exp(np.mean(scores)))
        
    return np.mean(split_scores)

# 3. Define `ImagePathDataset` class

In [3]:
class ImagePathDataset(Dataset):
    def __init__(self, files, transform=None):
        
        self.files = files
        self.transform = transform
        
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, i):
        path = self.files[i]
        img = Image.open(path).convert('RGB')
        if self.transform is not None:
            img = self.transform(img)
        return img

# 4. Calculate `Inception Score`

In [4]:
IMAGE_EXTENSIONS = ['jpg', 'png', 'jpeg', 'bmp', 'tiff', '.JPG']

In [6]:
%%time

# gen_imgs_dir = os.getcwd() + '/gen'
# print('Total images in gen_imgs_dir:', len(next(os.walk(gen_imgs_dir))[2]))
gen_imgs_dir = r'D:\GLAZE-evaluation\dataset\512-ori'
print ('Total files in gen_path: ', len(os.listdir(gen_imgs_dir)))

path = pathlib.Path(gen_imgs_dir)
files = sorted([file for ext in IMAGE_EXTENSIONS for file in path.glob('*.{}'.format(ext))])

dataset = ImagePathDataset(files, transform=transforms.ToTensor())
print("Inception Score (IS):", round(calculate_inception_score(dataset, batch_size=50, resize=True, splits=10), 3))

Total files in gen_path:  270
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to C:\Users\jul/.cache\torch\hub\checkpoints\inception_v3_google-0cc3c7bd.pth


100%|██████████| 104M/104M [00:33<00:00, 3.26MB/s] 


Inception Score (IS): 2.729
CPU times: total: 2min 45s
Wall time: 1min 2s


---

# Inception Score
- lora-ori = 1.676
- lora-glz = 1.997

In [5]:
%%time

# gen_imgs_dir = os.getcwd() + '/gen'
# print('Total images in gen_imgs_dir:', len(next(os.walk(gen_imgs_dir))[2]))
gen_imgs_dir = r'D:\GLAZE-evaluation\dataset\512-lora-ori'
print ('Total files in gen_path: ', len(os.listdir(gen_imgs_dir)))

path = pathlib.Path(gen_imgs_dir)
files = sorted([file for ext in IMAGE_EXTENSIONS for file in path.glob('*.{}'.format(ext))])

dataset = ImagePathDataset(files, transform=transforms.ToTensor())
print("Inception Score (IS):", round(calculate_inception_score(dataset, batch_size=50, resize=True, splits=10), 3))

Total files in gen_path:  200
Inception Score (IS): 1.676
CPU times: total: 2min 4s
Wall time: 22.4 s


In [6]:
%%time

# gen_imgs_dir = os.getcwd() + '/gen'
# print('Total images in gen_imgs_dir:', len(next(os.walk(gen_imgs_dir))[2]))
gen_imgs_dir = r'D:\GLAZE-evaluation\dataset\512-lora-glz'
print ('Total files in gen_path: ', len(os.listdir(gen_imgs_dir)))

path = pathlib.Path(gen_imgs_dir)
files = sorted([file for ext in IMAGE_EXTENSIONS for file in path.glob('*.{}'.format(ext))])

dataset = ImagePathDataset(files, transform=transforms.ToTensor())
print("Inception Score (IS):", round(calculate_inception_score(dataset, batch_size=50, resize=True, splits=10), 3))

Total files in gen_path:  200
Inception Score (IS): 1.997
CPU times: total: 1min 50s
Wall time: 19 s
